In [1]:
import sys
import os
sys.path.append(os.path.abspath('../')) # Point to project root

In [2]:
import importlib
import modules.trainer
import modules.utils
import modules.callbacks
import modules.env
import env.custom_hopper

# Force reload of the modules you changed
importlib.reload(modules.env)
importlib.reload(modules.callbacks)
importlib.reload(modules.utils)
importlib.reload(modules.trainer)
importlib.reload(env.custom_hopper)

# Re-import the specific functions into the global namespace
from modules.trainer import train_agent
from modules.utils import plot_learning_curve, plot_doraemon_dynamics
from env.custom_hopper import *

print("Modules reloaded successfully!")

Modules reloaded successfully!


/Users/alessandrobenvenuti/miniconda3/envs/RL_lab04/lib/python3.9/site-packages/gymnasium/envs/registration.py:644: UserWarning: WARN: Overriding environment CustomHopper-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/alessandrobenvenuti/miniconda3/envs/RL_lab04/lib/python3.9/site-packages/gymnasium/envs/registration.py:644: UserWarning: WARN: Overriding environment CustomHopper-source-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/alessandrobenvenuti/miniconda3/envs/RL_lab04/lib/python3.9/site-packages/gymnasium/envs/registration.py:644: UserWarning: WARN: Overriding environment CustomHopper-target-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [3]:
from modules.trainer import train_agent
from modules.utils import *
from env.custom_hopper import *

In [4]:
config = {
    'env_id': 'CustomHopper-source-v0',
    'algo': 'sac',
    'seed': 42,
    'timesteps': 1_000_000,
    'vectorize': True,
    'normalize': True, 
    'lr': 1e-3,
    
    # DORAEMON Settings
    'use_doraemon': True,     # This triggers GaussianHopperWrapper + DoraemonCallback

}

In [5]:
# Training
model, env, doraemon_cb = train_agent(config)

--- STARTING NEW TRAINING ---
Using cpu device
Logging to ./tensorboard_logs/SAC_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.8     |
|    ep_rew_mean     | 18.3     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 3726     |
|    time_elapsed    | 0        |
|    total_timesteps | 95       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21       |
|    ep_rew_mean     | 15.8     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 379      |
|    time_elapsed    | 0        |
|    total_timesteps | 168      |
| train/             |          |
|    actor_loss      | -4.97    |
|    critic_loss     | 0.385    |
|    ent_coef        | 0.936    |
|    ent_coef_loss   | -0.333   |
|    learning_rate   | 0.001    |
|    n_updates       | 67       |
---------------------------------

TypeError: Object of type float32 is not JSON serializable

In [8]:
# ONLY USE IF TRAINING WAS INTERRUPTED AND YOU HAVE CHECKPOINTS

import os
import re

# CHECKPOINT VERIFICATION

checkpoint_dir = "./logs/checkpoints"
if os.path.exists(checkpoint_dir):
    print("Files found:", os.listdir(checkpoint_dir))
else:
    print("No checkpoints found! The interrupt save didn't work.")


# RESUME TRAINING FROM CHECKPOINT

# 1. Find the latest checkpoint step
log_dir = "./logs/"
ckpt_dir = os.path.join(log_dir, "checkpoints")

latest_step = None
if os.path.exists(ckpt_dir):
    # regex to find numbers in filenames
    steps = [int(re.search(r'\d+', f).group()) for f in os.listdir(ckpt_dir) if "model_" in f]
    if steps:
        latest_step = max(steps)

print(f"Resuming from step: {latest_step}")

# 2. Resume Training
if latest_step is not None:
    # Pass the step number to resume_step
    model, env, doraemon_cb = train_agent(config, log_dir=log_dir, resume_step=latest_step)
else:
    print("Could not find a checkpoint to resume.")

Files found: ['doraemon_state_151660.json', 'doraemon_state_156701.json', 'vecnormalize_156701.pkl', 'vecnormalize_156311.pkl', 'doraemon_state_156311.json', 'doraemon_state_158913.json', 'replay_buffer_151660.pkl', 'model_156311.zip', 'model_156701.zip', 'replay_buffer_158913.pkl', 'model_151660.zip', 'replay_buffer_156701.pkl', 'replay_buffer_156311.pkl', 'model_158913.zip', 'vecnormalize_151660.pkl', 'doraemon_state_120805.json', 'vecnormalize_158913.pkl']
Resuming from step: 158913
--- RESUMING TRAINING FROM STEP 158913 ---
Loaded VecNormalize stats.
Loaded Model and Replay Buffer.
Restored DORAEMON: Lambda=1.52, Mean=[1.0999999046325684, 1.1999998092651367, 0.6999999284744263], Std=[0.01648724265396595, 0.01648724265396595, 0.01648724265396595]
Logging to ./tensorboard_logs/SAC_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 420      |
|    ep_rew_mean     | 1.53e+03 |
| time/              |          |
|    episodes        | 544      |


In [7]:
# Plot
# plot_learning_curve("./logs/")
plot_doraemon_dynamics(doraemon_cb) # uses the new function from utils.py

NameError: name 'doraemon_cb' is not defined

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

def evaluate_sim2real(model, source_env_raw, target_env_raw, log_dir, model_name, n_eval_episodes=20):
    """
    Evaluates the model on Source (Training Domain) vs Target (Shifted Domain).
    Crucially, it applies the SAME Normalization stats used during training.
    """
    # 1. Wrap raw envs into Vectorized Envs (required for SB3)
    source_vec = DummyVecEnv([lambda: source_env_raw])
    target_vec = DummyVecEnv([lambda: target_env_raw])

    # 2. Load Normalization Statistics
    # We must use the stats (mean/var) from the TRAINED agent, not new ones!
    norm_path = f"{log_dir}/{model_name}_vecnormalize.pkl"
    
    try:
        # Load the stats and apply them to the new envs
        source_vec = VecNormalize.load(norm_path, source_vec)
        target_vec = VecNormalize.load(norm_path, target_vec)
        
        # IMPORTANT: Turn off training updates and reward normalization for evaluation
        source_vec.training = False
        source_vec.norm_reward = False
        target_vec.training = False
        target_vec.norm_reward = False
        
        print(f"Loaded Normalization stats from {norm_path}")
    except FileNotFoundError:
        print("Warning: No normalization stats found. Evaluating with RAW observations (Expect poor performance).")

    # 3. Evaluate on Source (The "Sim")
    print("\n--- Evaluating on SOURCE Env (Simulation) ---")
    mean_reward, std_reward = evaluate_policy(model, source_vec, n_eval_episodes=n_eval_episodes, deterministic=True)
    print(f"Source Reward: {mean_reward:.2f} +/- {std_reward:.2f}")

    # 4. Evaluate on Target (The "Real")
    print("\n--- Evaluating on TARGET Env (Real/Shifted) ---")
    mean_reward_real, std_reward_real = evaluate_policy(model, target_vec, n_eval_episodes=n_eval_episodes, deterministic=True)
    print(f"Target Reward: {mean_reward_real:.2f} +/- {std_reward_real:.2f}")
    
    return mean_reward, mean_reward_real

# ---------------------------------------------------------
# EXECUTION
# ---------------------------------------------------------
# Define the raw environments
source_env = gym.make('CustomHopper-source-v0')
target_env = gym.make('CustomHopper-target-v0')

# Reconstruct the model name used in training
model_name = f"{config['algo']}_doraemon_{config['seed']}"

# Run evaluation
evaluate_sim2real(model, source_env, target_env, log_dir="./logs/", model_name=model_name)